In [1]:
import numpy as np
import h5py
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from collections.abc import Iterable

batchSize = 32 #Batch size of training set

#import helpers

In [2]:
def trainNetwork(model, loss_function, optimizer, numEpochs, dataloader, numInputs):
    #Function always assumes that intens column is first column
    #In the future, perhaps move the intensity processing to a different function?
    
    for epoch in range(numEpochs):
    
        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(dataloader, 0):

            # Get and prepare input
            totalColumns = data.size()[1]
            numOutputs = totalColumns - numInputs

            preProcessedInputs = data[:, 0:numInputs] #This line doesn't really do anything, delete later?
            targets = data[:, numInputs:(numInputs+numOutputs)]

            #Process intensity by putting it on a log scale
            intens = data[:, 0:1]
            intens = np.log(intens)
            inputs = torch.cat((intens, data[:,1:numInputs]), axis = 1)

            #Process targets by putting them on a log scale
            targets = np.log(targets)

            #print(type(inputs))

            #Comment the next two lines out if not using GPU
            inputs = inputs.to('cuda')
            targets = targets.to('cuda')

            #Normalize inputs
            inputs, targets = inputs.float(), targets.float()
            targets = targets.reshape((targets.shape[0], numOutputs))

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            inputs = inputs
            outputs = model(inputs)

            #The following two lines are for debugging only
    #         if i % 10 == 0:
    #             print("Targets:", targets[0:2])
    #             print("Outputs:", outputs[0:2])
    #             print()
    #             print()

            # Compute loss
            loss = loss_function(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 10 == 0:
                print('Loss after mini-batch %5d: %.3f' %
                     (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished.')

In [5]:
#See if I can import all of the data from a file

points = 20000

# filename = 'Dataset/Data_Fuchs_v_2.7_Wright_Pat_Narrow_Range_energy_limit_0.01_deviation_0.1_lambda_um_0.8_points_' \
#                 + str(points) + '_seed_0.h5'

filename = 'Dataset/Data_Fuchs_v_2.2_Wright_Pat_Narrow_Range_lambda_um_0.8_points_10000_seed_0.h5'

h5File = h5py.File(filename, 'r+')

In [6]:
#Read columns

intens = h5File['Intensity_(W_cm2)']

duration = h5File['Pulse_Duration_(fs)']

thickness = h5File['Target_Thickness (um)']

spotSize = h5File['Spot_Size_(FWHM um)']

maxEnergy = h5File['Max_Proton_Energy_(MeV)']

totalEnergy = h5File['Total_Proton_Energy_(MeV)']

avgEnergy = h5File['Avg_Proton_Energy_(MeV)']

test = zip(intens, duration, thickness, spotSize, maxEnergy, totalEnergy, avgEnergy)

print(next(test)) #Prints the first row from the h5 file

nextRow = next(test)

print(type(nextRow))

#Convert columns into numpy arrays
npIntens = np.fromiter(intens, float)
npDuration = np.fromiter(duration, float)
npThickness = np.fromiter(thickness, float)
npSpot = np.fromiter(spotSize, float)
npMaxEnergy = np.fromiter(maxEnergy, float)
npTotalEnergy = np.fromiter(totalEnergy, float)
npAvgEnergy = np.fromiter(avgEnergy, float)

#print(npIntens)

print(npIntens.shape)

#Join all of those arrays into one big numpy array
npFile = np.dstack((npIntens, npDuration, npThickness, npSpot, npMaxEnergy, npTotalEnergy, npAvgEnergy))

#Two input version
npFile = np.dstack((npIntens, npThickness, npMaxEnergy, npTotalEnergy, npAvgEnergy))

print(npFile.shape)

#npFile = npFile.reshape(10000, 7)

#Two input version
npFile = npFile.reshape(10000, 5)

print(npFile.shape)

print("Average Energy:", npAvgEnergy)
print("Total Energy:", npTotalEnergy)

print(npFile)

#npTrain = npFile[:9000, 0:7]
#npTest = npFile[9000:, 0:7]

npTrain = npFile[:9000, 0:5]
npTest = npFile[9000:, 0:5]

print(npTrain)
print(npTest)
print("First element:", npTrain[0])


(3.538453590906543e+18, 40.0, 4.225643132833221, 2.0, 0.010835492564865762, 1.4856161129848022e-08, 0.001689811439248253)
<class 'tuple'>
(10000,)
(1, 10000, 5)
(10000, 5)
Average Energy: [0.00168981 0.00903983 0.00080004 ... 0.00163991 0.0001079  0.0028896 ]
Total Energy: [1.48561611e-08 2.41391952e-07 5.61703127e-09 ... 2.29250478e-08
 8.74046945e-11 5.95510623e-08]
[[3.53845359e+18 4.22564313e+00 1.08354926e-02 1.48561611e-08
  1.68981144e-03]
 [5.19026302e+18 8.90987569e-01 6.24126123e-02 2.41391952e-07
  9.03983370e-03]
 [4.00648366e+18 9.27135540e+00 5.00937097e-03 5.61703127e-09
  8.00040532e-04]
 ...
 [5.73362811e+18 8.60258979e+00 1.03812396e-02 2.29250478e-08
  1.63990717e-03]
 [1.05630036e+18 7.47518077e+00 6.65338584e-04 8.74046945e-11
  1.07899169e-04]
 [6.50991143e+18 6.57047545e+00 1.85702376e-02 5.95510623e-08
  2.88959944e-03]]
[[3.53845359e+18 4.22564313e+00 1.08354926e-02 1.48561611e-08
  1.68981144e-03]
 [5.19026302e+18 8.90987569e-01 6.24126123e-02 2.41391952e-07
 

In [ ]:
print(npFile)

In [ ]:
#Check out one of the columns

intens[:]

#Can we convert intens by a log
logIntens = np.log(intens)

print(intens[:])
print(logIntens[:])

In [ ]:
#Print out every value in a column

# print(type(totalEnergy))

# for i in range(len(intens)):
#     print(intens[i])

In [7]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.norm0 = nn.BatchNorm1d(2)
    self.linear1 = nn.Linear(in_features=2, out_features=64)
    self.norm1 = nn.BatchNorm1d(64)
    self.act1 = nn.LeakyReLU()
    self.linear2 = nn.Linear(in_features=64, out_features=16)
    self.norm2 = nn.BatchNorm1d(16)
    self.act2 = nn.LeakyReLU()
    self.linear3 = nn.Linear(in_features=16, out_features=16)
    self.norm3 = nn.BatchNorm1d(16)
    self.act3 = nn.LeakyReLU()
    self.output = nn.Linear(in_features=16, out_features = 3)
    


  def forward(self, x):
    '''
      Forward pass
    '''
    x = self.norm0(x)
    x = self.linear1(x)
    x = self.norm1(x)
    x = self.act1(x)
    x = self.linear2(x)
    x = self.norm2(x)
    x = self.act2(x)
    x = self.output(x)
    
    
    return x

class MultiRegressor1Layer(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.norm0 = nn.BatchNorm1d(2)
    self.linear1 = nn.Linear(in_features=2, out_features=64)
    self.norm1 = nn.BatchNorm1d(64)
    self.act1 = nn.LeakyReLU()
    self.dropout = nn.Dropout()
    self.linear2 = nn.Linear(in_features=64, out_features=16)
    self.norm2 = nn.BatchNorm1d(16)
    #self.dropout = nn.Dropout()
    self.act2 = nn.LeakyReLU()
    self.output = nn.Linear(in_features=16, out_features = 3)
    


  def forward(self, x):
    '''
      Forward pass
    '''
    x = self.norm0(x)
    x = self.linear1(x)
    x = self.norm1(x)
    x = self.act1(x)
    #x = self.dropout(x)
    x = self.linear2(x)
    x = self.norm2(x)
    #x = self.dropout(x)
    x = self.act2(x)
    x = self.output(x)
    
    
    return x

In [8]:
#Load in the data
#Unfamiliar with h5py, Hopefully this doesn't break anything

#h5File.close()



#f = h5py.File('test.hdf5', 'w')
#f = h5py.File('test', 'r+')

training_dataset = h5File.create_dataset(name=None, data=npTrain)


#print(list(h5File.keys()))
#test = h5File[0:7]
#test = h5File['Intensity_(W_cm2)','Pulse_Duration_(fs)']


# Prepare our dataset

In [9]:
#Will the PyTorch DataLoader class work with H5Py datasets?

dataloader = DataLoader(training_dataset, batch_size=batchSize, shuffle=True)

In [10]:
iterDataLoader = iter(dataloader)

print(next(iterDataLoader))
print(next(iterDataLoader))

row = next(iterDataLoader)

tensor([[5.3120e+18, 2.3433e+00, 3.7639e-02, 1.2655e-07, 5.6370e-03],
        [2.9343e+18, 7.2526e+00, 4.0928e-03, 3.0952e-09, 6.5295e-04],
        [9.2354e+18, 2.1362e+00, 1.0447e-01, 9.1258e-07, 1.5034e-02],
        [1.3033e+18, 1.3505e+00, 4.7316e-03, 1.7626e-09, 7.3732e-04],
        [3.3213e+18, 1.7058e+00, 2.0882e-02, 3.4547e-08, 3.1632e-03],
        [5.3196e+18, 7.3999e+00, 1.1199e-02, 2.3620e-08, 1.7624e-03],
        [7.5263e+18, 7.1133e+00, 2.1546e-02, 8.5667e-08, 3.3478e-03],
        [3.0556e+18, 6.8610e+00, 4.7240e-03, 3.9547e-09, 7.5174e-04],
        [8.9581e+18, 6.1433e+00, 3.5145e-02, 2.0450e-07, 5.3831e-03],
        [8.7435e+18, 4.5280e+00, 4.8162e-02, 3.0637e-07, 7.2613e-03],
        [6.3940e+18, 7.8529e+00, 1.4226e-02, 4.0162e-08, 2.2322e-03],
        [4.5690e+18, 9.8781e+00, 5.7432e-03, 7.7972e-09, 9.1658e-04],
        [1.9463e+18, 9.7007e+00, 1.3376e-03, 4.1700e-10, 2.1643e-04],
        [3.6454e+18, 8.8777e+00, 4.5220e-03, 4.4156e-09, 7.2263e-04],
        [4.5765e+18,

In [11]:
# valueIntens, valueDuration, valueThickness, valueSpot, valueMax = row[0,0], row[0,1], row[0,2], row[0,3], row[0,4]

# print(valueIntens, valueDuration, valueThickness, valueSpot, valueMax)

# inputs = row[0, 0:4]


# print(inputs)

In [12]:
# #Test iterating through the data loader

# for i, data in enumerate(dataloader, 0):
#     #Print inputs
#     #print(data[0])
#     inputs = data[:, 0:4]
#     target = data[:, 4:7]
    
#     print("Iteration:", i)
#     print("Inputs:", inputs)
#     print("Target:", target)


In [13]:
# #Can we apply a log to just intens section of the inputs?

# copy = inputs
# intensCopy = row[:, 0:1]

# intensCopy = np.log(intensCopy)

# #print(intensCopy)

# #processedInput = 

# #Can we concatanate this with the other tensors?

# print("Size:", intensCopy.size())
# print("Size:", row[:,1:4].size())

# concatVer = torch.cat((intensCopy, row[:, 1:4]), axis = 1)

# print(concatVer)
# print(row[:,0:4])

In [14]:
#Initialize neural network

#model = MLP()
model = MLP().to('cuda') #GPU Version, comment out if not using GPU
#model = MultiRegressor1Layer().to('cuda')

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
numEpochs = 25

print(model)

trainNetwork(model, loss_function, optimizer, numEpochs, dataloader, numInputs = 2)

MLP(
  (norm0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=2, out_features=64, bias=True)
  (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=64, out_features=16, bias=True)
  (norm2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (norm3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (output): Linear(in_features=16, out_features=3, bias=True)
)
Starting epoch 1
Loss after mini-batch     1: 0.277
Loss after mini-batch    11: 2.648
Loss after mini-batch    21: 2.735
Loss after mini-batch    31: 2.685
Loss after mini-batch    41: 2.614
Loss after mini-batch    51: 2.619
Loss after mini-batch    61

Loss after mini-batch    61: 0.003
Loss after mini-batch    71: 0.005
Loss after mini-batch    81: 0.004
Loss after mini-batch    91: 0.005
Loss after mini-batch   101: 0.006
Loss after mini-batch   111: 0.005
Loss after mini-batch   121: 0.003
Loss after mini-batch   131: 0.004
Loss after mini-batch   141: 0.006
Loss after mini-batch   151: 0.004
Loss after mini-batch   161: 0.005
Loss after mini-batch   171: 0.004
Loss after mini-batch   181: 0.004
Loss after mini-batch   191: 0.005
Loss after mini-batch   201: 0.004
Loss after mini-batch   211: 0.003
Loss after mini-batch   221: 0.002
Loss after mini-batch   231: 0.003
Loss after mini-batch   241: 0.003
Loss after mini-batch   251: 0.004
Loss after mini-batch   261: 0.004
Loss after mini-batch   271: 0.004
Loss after mini-batch   281: 0.003
Starting epoch 9
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.006
Loss after mini-batch    21: 0.005
Loss after mini-batch    31: 0.006
Loss after mini-batch    41: 0.005
Los

Loss after mini-batch    41: 0.004
Loss after mini-batch    51: 0.004
Loss after mini-batch    61: 0.004
Loss after mini-batch    71: 0.003
Loss after mini-batch    81: 0.003
Loss after mini-batch    91: 0.003
Loss after mini-batch   101: 0.003
Loss after mini-batch   111: 0.002
Loss after mini-batch   121: 0.003
Loss after mini-batch   131: 0.003
Loss after mini-batch   141: 0.004
Loss after mini-batch   151: 0.007
Loss after mini-batch   161: 0.005
Loss after mini-batch   171: 0.003
Loss after mini-batch   181: 0.003
Loss after mini-batch   191: 0.003
Loss after mini-batch   201: 0.001
Loss after mini-batch   211: 0.002
Loss after mini-batch   221: 0.004
Loss after mini-batch   231: 0.004
Loss after mini-batch   241: 0.004
Loss after mini-batch   251: 0.002
Loss after mini-batch   261: 0.004
Loss after mini-batch   271: 0.002
Loss after mini-batch   281: 0.002
Starting epoch 17
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.003
Loss after mini-batch    21: 0.002
Lo

Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.002
Loss after mini-batch    51: 0.003
Loss after mini-batch    61: 0.004
Loss after mini-batch    71: 0.003
Loss after mini-batch    81: 0.002
Loss after mini-batch    91: 0.003
Loss after mini-batch   101: 0.003
Loss after mini-batch   111: 0.003
Loss after mini-batch   121: 0.004
Loss after mini-batch   131: 0.003
Loss after mini-batch   141: 0.003
Loss after mini-batch   151: 0.003
Loss after mini-batch   161: 0.002
Loss after mini-batch   171: 0.003
Loss after mini-batch   181: 0.002
Loss after mini-batch   191: 0.004
Loss after mini-batch   201: 0.003
Loss after mini-batch   211: 0.003
Loss after mini-batch   221: 0.003
Loss after mini-batch   231: 0.003
Loss after mini-batch   241: 0.003
Loss after mini-batch   251: 0.002
Loss after mini-batch   261: 0.003
Loss after mini-batch   271: 0.002
Loss after mini-batch   281: 0.004
Starting epoch 25
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.003
Lo

In [15]:
model.eval()

MLP(
  (norm0): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=2, out_features=64, bias=True)
  (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=64, out_features=16, bias=True)
  (norm2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (norm3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (output): Linear(in_features=16, out_features=3, bias=True)
)

In [16]:
# Specify a path
PATH = "test_regression.pt"

# Save
torch.save(model.state_dict(), PATH)

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

#Let's put in the training dataset in now

test_dataset = h5File.create_dataset(name=None, data=npTest)
testDataloader = DataLoader(test_dataset, batch_size=500, shuffle=True)
iterDataLoader = iter(testDataloader)
testData = next(iterDataLoader)

#Originally had tested with the full dataset
# fulldata = DataLoader(training_dataset, batch_size=5000, shuffle=True)
# iterDataLoader = iter(fulldata)
# allData = next(iterDataLoader)

print(testData)

#Get each separate input for future plotting reasons
intens = testData[:, 0:1]
duration = testData[:, 1]
thickness = testData[:, 2]
spotSize = testData[:, 3]

#Transform intens by a log function
logIntens = np.log(intens)

#print(logIntens.size(), allData[:,1:4].size())
#Process intensity by putting it on a log scale
inputs = torch.cat((logIntens, testData[:,1:4]), axis = 1)

#Turn logIntens back into a tensor that can be passed to a plot
intens = testData[:, 0]
logIntens = np.log(intens)

target = testData[:, 4]

inputs = inputs.to('cuda')
target = target.to('cuda')

inputs, target = inputs.float(), target.float()
target = target.reshape((target.shape[0], 1))

output = model(inputs)

#print(output)
print(output)
print(target)

tensor([[3.0222e+18, 4.2750e+00, 8.1394e-03, 8.4813e-09, 1.2759e-03],
        [2.2372e+18, 5.9951e+00, 3.2626e-03, 1.7244e-09, 5.2049e-04],
        [5.4289e+18, 9.3179e+00, 8.4318e-03, 1.6109e-08, 1.3378e-03],
        ...,
        [1.2727e+18, 1.6589e+00, 4.0307e-03, 1.3762e-09, 6.3101e-04],
        [6.6212e+18, 9.7405e+00, 1.1166e-02, 2.9553e-08, 1.7655e-03],
        [9.0304e+18, 6.1465e+00, 3.5614e-02, 2.1014e-07, 5.4531e-03]],
       dtype=torch.float64)


RuntimeError: running_mean should contain 4 elements not 2

In [ ]:
#Plot just actual data itself

print(target.size())
print(intens.size())
print(output.size())
print(intens[0])

#Max KE Energy plots

fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens,target[:].cpu().detach().numpy(), color = 'blue')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 2)
plt.scatter(duration,target[:].cpu().detach().numpy(), color = 'blue')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 3)
plt.scatter(thickness,target[:].cpu().detach().numpy(), color = 'blue')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 4)
plt.scatter(spotSize,target[:].cpu().detach().numpy(), color = 'blue')
plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')
plt.tight_layout()
plt.show()

In [ ]:
#Plot the actual data and the regressor's predicted data

#Max KE Energy plots

fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens,target[:].cpu().detach().numpy(), color = 'blue')
plt.scatter(logIntens,np.exp(output.cpu().detach().numpy()), color = 'red')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 2)
plt.scatter(duration,target[:].cpu().detach().numpy(), color = 'blue')
plt.scatter(duration,np.exp(output.cpu().detach().numpy()), color = 'red')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 3)
plt.scatter(thickness,target[:].cpu().detach().numpy(), color = 'blue')
plt.scatter(thickness,np.exp(output.cpu().detach().numpy()), color = 'red')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 4)
plt.scatter(spotSize,target[:].cpu().detach().numpy(), color = 'blue')
plt.scatter(spotSize,np.exp(output.cpu().detach().numpy()), color = 'red')
plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')
plt.tight_layout()
plt.show()

# Error Plots

In [ ]:
#Plot relative error

difference = target[:].cpu().detach().numpy() - np.exp(output.cpu().detach().numpy())
error = np.divide(difference, np.exp(output.cpu().detach().numpy())) * 100

fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens, error, color = 'blue')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 2)
plt.scatter(duration,error, color = 'blue')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 3)
plt.scatter(thickness,error, color = 'blue')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 4)
plt.scatter(spotSize,error, color = 'blue')
plt.ylabel('Max KE Error (%)')
plt.ylabel('Error (%)')
plt.tight_layout()
plt.show()

In [ ]:
#Plot relative error magnitude

difference = np.abs(target[:].cpu().detach().numpy() - np.exp(output.cpu().detach().numpy()))
error = np.divide(difference, np.exp(output.cpu().detach().numpy())) * 100

fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens, error, color = 'blue')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 2)
plt.scatter(duration,error, color = 'blue')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 3)
plt.scatter(thickness,error, color = 'blue')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max KE Error (%)')

plt.subplot(1, 4, 4)
plt.scatter(spotSize,error, color = 'blue')
plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
plt.ylabel('Max KE Error (%)')
plt.tight_layout()
plt.show()

In [ ]:
#Also plot absolute errors

difference = target[:].cpu().detach().numpy() - np.exp(output.cpu().detach().numpy())


fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens, difference, color = 'blue')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 2)
plt.scatter(duration, difference, color = 'blue')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 3)
plt.scatter(thickness, difference, color = 'blue')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 4)
plt.scatter(spotSize, difference, color = 'blue')
plt.ylabel('Max KE Error')
plt.ylabel('Error (%)')
plt.tight_layout()
plt.show()

In [ ]:
#Absolute error magnitude

difference = np.abs(target[:].cpu().detach().numpy() - np.exp(output.cpu().detach().numpy()))


fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens, difference, color = 'blue')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 2)
plt.scatter(duration, difference, color = 'blue')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 3)
plt.scatter(thickness, difference, color = 'blue')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max KE Error')

plt.subplot(1, 4, 4)
plt.scatter(spotSize, difference, color = 'blue')
plt.ylabel('Max KE Error')
plt.ylabel('Error (%)')
plt.tight_layout()
plt.show()

# Could be better, but we're just using this regression neural network as practice for when we make an invertible regression neural network. There seems to be a small habit of underestimating the actual values.

## The regresion neural network looks like it does a pretty good job of predicting the max proton energy values, now can we get it to predict all three values at once?

In [ ]:
#Define a new neural network that outputs three features rather than one

class MultiRegressor(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.norm0 = nn.BatchNorm1d(4)
    self.linear1 = nn.Linear(in_features=4, out_features=64)
    self.norm1 = nn.BatchNorm1d(64)
    self.act1 = nn.LeakyReLU()
    self.dropout = nn.Dropout()
    self.linear2 = nn.Linear(in_features=64, out_features=16)
    self.norm2 = nn.BatchNorm1d(16)
    #self.dropout = nn.Dropout()
    self.act2 = nn.LeakyReLU()
    self.output = nn.Linear(in_features=16, out_features = 3)
    


  def forward(self, x):
    '''
      Forward pass
    '''
    x = self.norm0(x)
    x = self.linear1(x)
    x = self.norm1(x)
    x = self.act1(x)
    #x = self.dropout(x)
    x = self.linear2(x)
    x = self.norm2(x)
    #x = self.dropout(x)
    x = self.act2(x)
    x = self.output(x)
    
    
    return x

## The following two cells were used by me to figure out some stuff about the shape of our outputs

In [ ]:
targets = data[:, 4:7]

#print(targets)

targets = data[:, 4]
print(targets.shape[0])

targets = data[:, 4:7]
print(targets.shape[0])

In [ ]:
targets = data[:, 4:7]
targets = targets.reshape((targets.shape[0], 3))
print(targets)

# Train our multi-output neural network

In [ ]:
#Initialize neural network

#model = MultiRegressor()
multiModel = MultiRegressor().to('cuda') #GPU Version, comment out if not using GPU

loss_function = nn.MSELoss()
#loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(multiModel.parameters(), lr=1e-3)
numEpochs = 20

print(multiModel)


trainNetwork(multiModel, loss_function, optimizer, numEpochs, dataloader, 3)

In [ ]:
#Validate with test dataset

testDataloader = DataLoader(test_dataset, batch_size=5000, shuffle=True)
iterDataLoader = iter(testDataloader)
testData = next(iterDataLoader)


#Old code utilizing the full dataset

# fulldata = DataLoader(training_dataset, batch_size=5000, shuffle=True)
# iterDataLoader = iter(fulldata)
# allData = next(iterDataLoader)

# print(allData)

#Get each separate input for future plotting reasons
intens = testData[:, 0:1]
duration = testData[:, 1]
thickness = testData[:, 2]
spotSize = testData[:, 3]

#Transform intens by a log function
logIntens = np.log(intens)

print(logIntens.size(), testData[:,1:4].size())
#Process intensity by putting it on a log scale
inputs = torch.cat((logIntens, testData[:,1:4]), axis = 1)

#Turn logIntens back into a tensor that can be passed to a plot
intens = testData[:, 0]
logIntens = np.log(intens)

#inputs = allData[:, 0:4]
target = testData[:, 4:7]
target = np.log(target)

inputs = inputs.to('cuda')
target = target.to('cuda')

inputs, target = inputs.float(), target.float()
target = target.reshape((target.shape[0],3))

#print("Inputs:", inputs)
#print("Targets:", target)

output = multiModel(inputs)

target = np.log(testData[:, 4:7])

print("Targets:", target)
#print(output)
print("Outputs:", output)

In [ ]:
print("Target:", target[:])
print(target[:, 0])
print("Output:", output)
print(output[:, 0])

## It seems the multi-regressor has a habit of under-predicting max and total proton energy by a slight margin. It is way off for average proton energy though, maybe try applying a log-scale?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#Plot the actual data and the regressor's predicted data

#Max KE Energy plots

index = 0

fig=plt.figure(figsize=(12,4))
plt.subplot(1, 4, 1)
plt.scatter(logIntens,np.exp(target[:, index].cpu().detach().numpy()), color = 'blue')
plt.scatter(logIntens,np.exp(output[:, index].cpu().detach().numpy()), color = 'red')
plt.xscale('log')
plt.xlabel(r'Intensity (W cm$^{-2}$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 2)
plt.scatter(duration,np.exp(target[:, index].cpu().detach().numpy()), color = 'blue')
plt.scatter(duration,np.exp(output[:, index].cpu().detach().numpy()), color = 'red')
plt.xlabel('Pulse Duration (fs)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 3)
plt.scatter(thickness,np.exp(target[:, index].cpu().detach().numpy()), color = 'blue')
plt.scatter(thickness,np.exp(output[:, index].cpu().detach().numpy()), color = 'red')
plt.xlabel(r'Target Thickness ($\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')

plt.subplot(1, 4, 4)
plt.scatter(spotSize,np.exp(target[:, index].cpu().detach().numpy()), color = 'blue')
plt.scatter(spotSize,np.exp(output[:, index].cpu().detach().numpy()), color = 'red')
plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
plt.ylabel('Max Proton Energy (MeV)')
plt.tight_layout()
plt.show()

# Error plots
## First max kinetic energy, then total energy, and finally average energy

In [ ]:
def plot_relative_error(target, output, logIntens, duration, thickness, spotSize, index, label):
    difference = np.exp(target[:, index].cpu().detach().numpy()) - np.exp(output[:, index].cpu().detach().numpy())
    difference = np.abs(difference)
    error = np.divide(difference, np.exp(output[:, index].cpu().detach().numpy())) * 100

    fig=plt.figure(figsize=(12,8))

    plt.subplot(1, 4, 1)
    plt.scatter(logIntens, error, color = 'blue')
    plt.xscale('log')
    #plt.ylim([0, 100])
    plt.xlabel(r'Intensity (W cm$^{-2}$)')
    plt.ylabel(label)

    plt.subplot(1, 4, 2)
    plt.scatter(duration, error, color = 'blue')
    plt.xlabel('Pulse Duration (fs)')
    plt.ylabel(label)

    plt.subplot(1, 4, 3)
    plt.scatter(thickness,error, color = 'blue')
    plt.xlabel(r'Target Thickness ($\mu m$)')
    plt.ylabel(label)
    
    plt.subplot(1, 4, 4)
    plt.scatter(spotSize, error, color = 'blue')
    plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
    plt.ylabel(label)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Positive absolute error means the NN had over-estimated the error
# Negative absolute error means the NN had under-estimated the error
def plot_absolute_error(target, output, logIntens, duration, thickness, spotSize, index, label):
    difference = np.exp(output[:, index].cpu().detach().numpy()) - np.exp(target[:, index].cpu().detach().numpy())
    absDifference = np.abs(difference)

    fig=plt.figure(figsize=(12,8))
    plt.subplot(2, 4, 1)
    plt.scatter(logIntens, difference, color = 'blue')
    plt.xscale('log')
    #plt.ylim([0, -1])
    plt.xlabel(r'Intensity (W cm$^{-2}$)')
    plt.ylabel(label)

    plt.subplot(2, 4, 2)
    plt.scatter(duration, difference, color = 'blue')
    plt.xlabel('Pulse Duration (fs)')
    plt.ylabel(label)

    plt.subplot(2, 4, 3)
    plt.scatter(thickness, difference, color = 'blue')
    plt.xlabel(r'Target Thickness ($\mu m$)')
    plt.ylabel(label)

    plt.subplot(2, 4, 4)
    plt.scatter(spotSize, difference, color = 'blue')
    plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
    plt.ylabel(label)

    plt.subplot(2, 4, 5)
    plt.scatter(logIntens, absDifference, color = 'blue')
    plt.xscale('log')
    #plt.ylim([0, 1])
    plt.xlabel(r'Intensity (W cm$^{-2}$)')
    plt.ylabel(label + " (Magnitude)")

    plt.subplot(2, 4, 6)
    plt.scatter(duration, absDifference, color = 'blue')
    plt.xlabel('Pulse Duration (fs)')
    plt.ylabel(label + " (Magnitude)")

    plt.subplot(2, 4, 7)
    plt.scatter(thickness, absDifference, color = 'blue')
    plt.xlabel(r'Target Thickness ($\mu m$)')
    plt.ylabel(label + " (Magnitude)")

    plt.subplot(2, 4, 8)
    plt.scatter(spotSize, absDifference, color = 'blue')
    plt.xlabel(r'Spot Size  (FWHM, $\mu m$)')
    plt.ylabel(label + " (Magnitude)")

    plt.tight_layout()
    plt.show()

In [ ]:
def calc_MSE_Error(target, output, index):
    result = np.square(np.subtract(np.exp(target[:, index].cpu().detach().numpy()), np.exp(output[:, index].cpu().detach().numpy())).mean())
                       
    return result

In [ ]:
def calc_Avg_Percent_Error(target, output, index):
    difference = np.exp(target[:, index].cpu().detach().numpy()) - np.exp(output[:, index].cpu().detach().numpy())
    difference = np.abs(difference)
    error = np.divide(difference, np.exp(output[:, index].cpu().detach().numpy())) * 100
    
    result = error.mean()
    
    return result

In [ ]:
#Plot relative error and relative error magnitude

plot_relative_error(target, output, logIntens, duration, thickness, spotSize, 0, 'Max KE Error (%)')

In [ ]:
#Also plot absolute errors

plot_absolute_error(target, output, logIntens, duration, thickness, spotSize, 0, 'Max KE Error')

In [ ]:
#Errors for total proton energy

plot_relative_error(target, output, logIntens, duration, thickness, spotSize, 1, 'Total Energy Error (%)')

In [ ]:
plot_absolute_error(target, output, logIntens, duration, thickness, spotSize, 1, 'Total Energy Error')

In [ ]:
#Finally, do average energy

plot_relative_error(target, output, logIntens, duration, thickness, spotSize, 2, 'Avg Energy Error (%)')

In [ ]:
plot_absolute_error(target, output, logIntens, duration, thickness, spotSize, 2, 'Avg Energy Error')

In [ ]:
#Calculate mean squared error values
#Also calculate the average relative error

error = [0., 1., 2.]
percentError = [0., 1., 2.]

for index in range(3):
    error[index] = calc_MSE_Error(target, output, index)
    percentError[index] = calc_Avg_Percent_Error(target, output, index)
    
print("Max KE MSE:", error[0])
print("Max KE Average Relative Error:", percentError[0])
print("Total Energy MSE:", error[1])
print("Total Energy Average Relative Error:", percentError[1])
print("Avg Energy MSE:", error[2])
print("Avg Energy Average Relative Error:", percentError[2])

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

#We'll be using matplotlib notebooks so we can easily rotate the 3-D plots interactively
#Note that when we have many, many points, these plots can be very slow to respond to rotations

size3D = (10,6) #Controls the figure size for our plots

fig = plt.figure(figsize=size3D)

#Intensity and Pulse Duration
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.mouse_init()
ax.set_xlabel('Max Energy', fontweight ='bold')
ax.set_ylabel('Total Energy', fontweight ='bold')
ax.set_zlabel('Average Energy', fontweight ='bold')
ax.scatter3D(target[:, 0].cpu().detach().numpy(), target[:, 1].cpu().detach().numpy(), target[:, 2].cpu().detach().numpy(),
            color = 'blue')
ax.scatter3D(output[:, 0].cpu().detach().numpy(), output[:, 1].cpu().detach().numpy(), output[:, 2].cpu().detach().numpy(),
            color = 'red')

# Conclusion:
- Apply the log function to intensity for neural networks and all of the outputs for the energies
- Maybe look into whether or not we only need to apply a log to the average energy?